# Fungut

# 00 Packages and Directory

In [13]:
import IPython

import pandas as pd
import matplotlib.pyplot as plt
import qiime2 as q2
import seaborn as sns
from qiime2 import Visualization


import os

import matplotlib.pyplot as plt
%matplotlib inline

# 01 Data import

In [16]:
PATH = "final/fungut_data/fungut_metadata.tsv"

In [17]:
surveys_df = pd.read_csv(PATH, sep="\t")

In [19]:
data_dir = 'final/fungut_data'

In [20]:
!qiime tools peek $data_dir/fungut_forward_reads.qza

UUID:        3638611d-1767-413b-9390-70ee3d78e4ff
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [21]:
!qiime demux summarize \
  --i-data $data_dir/fungut_forward_reads.qza \
  --o-visualization $data_dir/demux_summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: final/fungut_data/demux_summary.qzv


In [22]:
Visualization.load(f"{data_dir}/demux_summary.qzv")

<visualization: Visualization uuid: f3cf44e4-fbc6-4d58-8edc-c3ecd63e6329>

# 02 Denoising

In [23]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/rep_seqs.qza \
    --o-table $data_dir/table.qza \
    --o-denoising-stats $data_dir/stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
^C


In [ ]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
   --p-trim-left 0 \
   --p-trunc-len 160 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/rep_seqs.qza \
    --o-table $data_dir/table.qza \
    --o-denoising-stats $data_dir/stats.qza

# 03 Taxonomy